In [1]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

In [2]:
outpath = "/scratch/PI/horence/JuliaO/single_cell/Differential_Splicing/notebooks/output/significance_tables/"

In [45]:
datanames = {"TSP1_10x_with_postprocessing_nopanc_cellann" : "scZ","TSP2_10x_rerun_with_postprocessing_3prime_cellann" : "scZ",
            "TS_pilot_smartseq_with_postprocessing_nopanc_cellann" : "svd_z0", "TSP2_SS2_RUN1_RUN2_cellann" : "svd_z0",
            "TS_pilot_smartseq_with_postprocessing_nopanc_cellann_shared" : "scZ", "TSP2_SS2_RUN1_RUN2_cellann_shared" : "scZ",
             "TSP1_10x_with_postprocessing_nopanc_cellann_sharedjuncs_sharedonts_10x" : "scZ",
             "TSP2_10x_rerun_with_postprocessing_3prime_cellann_sharedjuncs_sharedonts_10x" : "scZ",
            "Lemur_10x_Stumpy_with_postprocessing_cellann" : "scZ", "Lemur_10x_Antoine_with_postprocessing_cellann" : "scZ",
            "Tabula_muris_senis_P1_10x_with_postprocessing_cellann": "scZ", "Tabula_muris_senis_P2_10x_with_postprocessing_cellann" : "scZ",
            "TSP1_10x_with_postprocessing_nopanc_cellann_sharedjuncs_sharedonts" : "scZ", "TSP2_10x_rerun_with_postprocessing_3prime_cellann_sharedjuncs_sharedonts" : "scZ",
            "TS_pilot_smartseq_with_postprocessing_nopanc_cellann_sharedjuncs_sharedonts" : "scZ", "TSP2_SS2_RUN1_RUN2_cellann_sharedjuncs_sharedonts" : "scZ"}

keys = list(datanames.keys())
# for key in keys:
#   datanames[key + "_shuffle"] = datanames[key]
var_path = "../data/"

# group = "ontology"
group = "compartment"
sub = "dummy"
within_col = "tiss_comp"


if group == "ontology":
  within_thresh = 0.5
elif group == "compartment":
  within_thresh = 0

zdict = {"scZ" : .5, "svd_z0" : 3.5}
# zdict = {"scZ" : 0, "svd_z0" : 0}

alpha = 0.05

In [44]:


sg_dict = {}
sgdf_dict = {}
subsg_dict = {}

zscore_path = "../data/"


out = {"dataname" : [],"num_pairs_sig" : [], "frac_pairs_sig" : [], "num_genes_sig" : [], "num_genes" : [], "frac_genes_sig" : [],
      "col_used" : [],"shuffled" : []}
for dataname, z_col in tqdm(datanames.items()):

  if os.path.isfile("{}{}_pvals_{}-{}_100_S_0.1_z_0.0_b_5_sig.tsv".format(var_path,dataname,group,sub)):
    df = pd.read_csv("{}{}_pvals_{}-{}_100_S_0.1_z_0.0_b_5_sig.tsv".format(var_path,dataname,group,sub),sep="\t")
  else:
    df = pd.read_csv("{}{}_pvals_{}-{}_100_S_0.1_z_0.0_b_5.tsv".format(var_path,dataname,group,sub),sep="\t")
    df1 = pd.read_parquet("{}{}_sym_SVD_normdonor_S_0.1_z_0.0_b_5.pq".format(zscore_path,dataname),columns=[z_col,"cell","geneR1A_uniq","ontology","cell_gene","tissue","compartment"])
    df1 = df1.drop_duplicates("cell_gene")
    df1["ontology_gene"] = df1[group] + df1["geneR1A_uniq"]
    df1["med_scZ"] = df1["ontology_gene"].map(df1.groupby('ontology_gene')[z_col].median())
    df1 = df1.drop_duplicates("ontology_gene")

    # find max and min within either tiss comp or just tissue
    df1["tiss_comp_gene"] = df1["tissue"] + df1["compartment"] + df1["geneR1A_uniq"]
    df1["tissue_gene"] = df1["tissue"] + df1["geneR1A_uniq"]

    df1["min_med"] = df1.groupby(within_col + "_gene")["med_scZ"].transform("min")
    df1["max_med"] = df1.groupby(within_col + "_gene")["med_scZ"].transform("max")
    df1["diff"] = df1["max_med"] - df1["min_med"]
    df1["gene_max_diff"] = df1.groupby("geneR1A_uniq")["diff"].transform("max")
#     df1 = df1[df1["gene_max_diff"] > within_thresh]
    df["gene_max_diff"] = df["geneR1A_uniq"].map(pd.Series(df1.gene_max_diff.values,index=df1.geneR1A_uniq).to_dict())
  df["sig"] = False
  df.loc[(df["perm_pval_adj_" + z_col] < alpha) & (df["max_abs_median_" + z_col] > zdict[z_col]),"sig"] = True
  df.loc[df["gene_max_diff"] < within_thresh,"sig"] = False
  df.to_csv("{}{}_pvals_{}-{}_100_S_0.1_z_0.0_b_5_sig.tsv".format(var_path,dataname,group,sub),sep="\t",index=False)


  out["dataname"].append(dataname)
  
  pairs = df["sig"].sum()
  out["num_pairs_sig"].append(pairs)
  subsg_dict[dataname] = (set(df[df["sig"]]["sub_col"]))

  try:
    out["frac_pairs_sig"].append(pairs/df.shape[0])
  except:
    out["frac_pairs_sig"].append(np.nan)
  
  genes = df[df["sig"]]["geneR1A_uniq"].nunique()
  df = df.sort_values("max_abs_median_" + z_col,ascending=False).reset_index(drop=True)
  sg_dict[dataname] = set(df[df["sig"]]["geneR1A_uniq"].unique())
  sgdf_dict[dataname] = df[["geneR1A_uniq","sub_col"]]
  out["num_genes_sig"].append(genes)
  out["num_genes"].append(df["geneR1A_uniq"].nunique())
  try:
    out["frac_genes_sig"].append(genes/df["geneR1A_uniq"].nunique())
  except:
    out["frac_genes_sig"].append(np.nan)
  out["col_used"].append(z_col)
  if dataname.endswith("shuffle"):
    out["shuffled"].append(True)
  else:
    out["shuffled"].append(False)
out_df = pd.DataFrame.from_dict(out)
out_df.to_csv("{}{}_{}_sig_{}_{}.tsv".format(outpath,group,sub,within_col,within_thresh),index=False,sep="\t")
out_df

100%|██████████| 16/16 [00:26<00:00,  1.66s/it]


,dataname,num_pairs_sig,frac_pairs_sig,num_genes_sig,num_genes,frac_genes_sig,col_used,shuffled
0,TSP1_10x_with_postprocessing_nopanc_cellann,24,0.031455,24,763,0.031455,scZ,False
1,TSP2_10x_rerun_with_postprocessing_3prime_cellann,22,0.016260,22,1353,0.016260,scZ,False
2,TS_pilot_smartseq_with_postprocessing_nopanc_c...,47,0.006243,47,7529,0.006243,svd_z0,False
3,TSP2_SS2_RUN1_RUN2_cellann,122,0.010273,122,11876,0.010273,svd_z0,False
4,TS_pilot_smartseq_with_postprocessing_nopanc_c...,147,0.021416,147,6864,0.021416,scZ,False
5,TSP2_SS2_RUN1_RUN2_cellann_shared,388,0.036711,388,10569,0.036711,scZ,False
6,TSP1_10x_with_postprocessing_nopanc_cellann_sh...,24,0.032432,24,740,0.032432,scZ,False
7,TSP2_10x_rerun_with_postprocessing_3prime_cell...,17,0.018221,17,933,0.018221,scZ,False
8,Lemur_10x_Stumpy_with_postprocessing_cellann,71,0.047973,71,1480,0.047973,scZ,False
9,Lemur_10x_Antoine_with_postprocessing_cellann,252,0.120459,252,2092,0.120459,scZ,False
